In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.ml.model import AlphaPolicyModel, AlphaValueModel
from src.connect4.agent import RandomAgent, MCTSAgent
from src.connect4.game import run
import time
from src.ml.train_util import record_tree
import tensorflow as tf

ImportError: cannot import name 'AlphaPolicyModel' from 'src.ml.model' (/Users/alacwong/Desktop/personal/connect4/connect4-ai/src/ml/model.py)

In [ ]:
alpha_value = AlphaValueModel()
alpha_policy = AlphaPolicyModel()

agent_1 = MCTSAgent(
    value_network=alpha_value,
    policy_network=alpha_policy
)

agent_2 = RandomAgent()

start = time.time()
run(agent_1, agent_2)
print(f' {time.time() - start} s')

In [ ]:
import tensorflow as tf

device = tf.test.gpu_device_name()
print(device)

In [ ]:
gpu = '/device:GPU:0'
cpu = '/device:CPU:0'

from src.ml.nn import get_value_network
import numpy as np

data = np.zeros((1,7 *6,))
model = get_value_network()


start = time.time()
with tf.device(cpu):
    res = model(data).numpy()[0][0]
    print(res, f'CPU: {time.time() - start}')

start = time.time()
with tf.device(gpu):
    res = model(data).numpy()[0][0]
    print(res, f'GPU: {time.time() - start}')
    

In [ ]:
    import time
    
    # See how well model fits self-play data
    with open('src/ml.generated/datasets/large-data.pkl', 'rb') as f:
        data = pickle.load(f)
        priors = data['priors']
        states = data['states']
        values = data['values']
    
    print(states.shape)
    start = time.time()
    states, priors, values = preprocess_data(states, priors, values)
    with open('src/ml/generated/datasets/cleaned_data.pkl', 'wb+') as f:
        pickle.dump(
            {
                'priors': priors,
                'states': states,
                'values': values
            }, f
        )
    print(f'Aggregate and process data in {time.time() - start}s')
    print(states.shape)

In [ ]:
    # See how well model performs on cleaned data
    from src.ml.nn import get_cnn_policy_model, get_cnn_value_model, get_value_network, get_policy_network
    import pickle
    from constants import col, row
    
    with open('src/ml/generated/datasets/cleaned_data.pkl', 'rb+')as f:
        data = pickle.load(f)
        priors = data['priors']
        states = data['states']
        values = data['values']
    
    print(priors.shape)

    value_network = get_cnn_value_model()
    
    print(value_network.summary())
    N, D = states.shape
    states = states.reshape((N, col, row, 1))
    policy_network = get_policy_network()
    print(policy_network.summary())
    #
    value_network.fit(states, values, epochs=10, validation_split=0.3)
    policy_network.fit(states.reshape(N, col* row), priors, epochs=10, validation_split=0.3)

In [ ]:
    # See how well model performs on cleaned data
    with open('generated/datasets/cleaned_data.pkl', 'rb+')as f:
        data = pickle.load(f)
        priors = data['priors']
        states = data['states']
        values = data['values']

    value_network = get_cnn_value_model()
    print(value_network.summary())
    N, D = states.shape
    states = states.reshape((N, col, row, 1))
    # policy_network = get_policy_network()

    #
    value_network.fit(states, values, epochs=50, validation_split=0.3)
    # policy_network.fit(states, priors, epochs=10, validation_split=0.3)